# Analysis of MD Data with MDTraj

You have already made use of the Python package MDTraj when you did the MD prearation workshop earlier in the week. Here you take a slightly deeper dive into it, so you can see how it can be used to analyse simulation data.

Start by importing the required packages:

In [ ]:
import mdtraj as mdt
from matplotlib import pyplot as plt
import nglview as nv

Now load the 50ns trajectory file:

In [ ]:
traj = mdt.load('/data/abl-ligand-md.nc', top='../MD-Analysis/abl_ligand.prmtop')

`traj` is our trajectory as an instance of an `MDTrajectory`. It has some useful attributes and methods, a very few of which we will explore here.

First you can find out a little about what it represents by `print`ing i:

In [ ]:
print(t)

It contains a `topolgy` object, that describes aspects of the system that are time-invariant:

In [ ]:
print(t.topology)

`MDTraj` has many simulation analysis methods let's start by lookling at RMSD calculation. We calculate the rmsd relative to the structure in the first frame of the trajectory. Notice how we use the simulation time, stored in the `traj` object, to scale our x-axis:

In [ ]:
rmsd = mdt.rmsd(t, t[0])
plt.plot(t.time, rmsd)
plt.xlabel('time (ps)')
plt.ylabel('RMSD (nm)')

Does this look odd to you? It's because we have calculated the RMSD over ALL atoms in each frame of the trajectory, so that includes the water molecules - not what we want!

MDTraj has a special syntax to allow you to make selections of subsets of the atoms to do calculations on. See [here](https://mdtraj.readthedocs.io/en/latest/atom_selection.html) for details. Here we do the RMSD considering just protein C-alpha atoms:

In [ ]:
ca_atoms = t.topology.select('name CA')
rmsd_ca = mdt.rmsd(t, t[0], atom_indices=ca_atoms)
plt.plot(t.time, rmsd_ca)
plt.xlabel('time (ps)')
plt.ylabel('RMSD (nm)')

Now let's look at the ligand:

In [ ]:
ligand_atoms = t.topology.select('resname UNL')
rmsd_lig = mdt.rmsd(t, t[0], atom_indices=ligand_atoms)
plt.plot(t.time, rmsd_lig)

It looks like something is going on - what? Lets visualize the trajectory. Before doing this, we will create a new trajectory file, containing just ligand atoms:

In [ ]:
ligand_traj = t.atom_slice(ligand_atoms)
view = nv.show_mdtraj(ligand_traj)
view

It's a little hard to se what's going on here because the ligand is shifting around. This global rotation and translation is not so interesting, we can remove it by superimposing every snapshot on the first:

In [ ]:
ligand_traj.superpose(ligand_traj[0])
view = nv.show_mdtraj(ligand_traj)
view

Now its a bit easier to se what's going on, but not super-obvious. Time for some measurments. We will calculate the distance between the N atom in the pyridine ring ('N1') and the terminal N atom in the piperidine ring ('N51'):

In [ ]:
atom_pair = ligand_traj.topology.select('name N1 or name N51')
n1_n51_distance = mdt.compute_distances(ligand_traj, [atom_pair])
plt.plot(ligand_traj.time, n1_n51_distance)

We can clearly see a major 'jump' in the distance around 33 nanoseconds, i.e. a major change in the conformation of the ligand (despite the fact that it is bound to the protein)

You could now look into this is much more detail - but that's enough for now.